<img src="../fasp/runner/credits/images/GTEXExample.jpg" style="float: right;">

### GTEX Example using PFB in BigQuery
This notebook computes on the freely available GTEX version 8 files on Amazon AWS using the Seven Bridges Cancer Genomics Cloud WES service. 

In this case the DRS ids of the GTEX files are available via a client which acccess a local manifest file. The manifest file is obtained from Anvil as follows.
https://anvilproject.org/learn/reference/gtex-v8-free-egress-instructions

Rather than use the Gen3 Downloader described in the egress instructions, this example uses the Anvil DRS service to obtain URLs which can be passed to the Seven Bridges CGC WES Service.

The following step demonstrates a query run against that file. 

In [1]:
from fasp.runner import FASPRunner

# The implementations we're using
from fasp.search import BigQuerySearchClient
from fasp.loc import anvilDRSClient
from fasp.workflow import sbcgcWESClient

faspRunner = FASPRunner(program='GTEXExample')

# Step 1 - Discovery
# query for relevant DRS objects
searchClient = BigQuerySearchClient()
query = '''select su.id subject_id, sq.object_id drsID 
from `isbcgc-216220.gtex8.sequencing` sq 
join `isbcgc-216220.gtex8.sample` sa on sq.parent_id = sa.id 
join `isbcgc-216220.gtex8.subject` su on sa.parent_id = su.id 
where su.body_mass_index between 22.0 and 25.0 
and data_type = 'Aligned Reads' and data_format = 'bam'
limit 3'''
res = searchClient.runQuery(query)
res

[Row(('1653e867-769a-4f4b-a17a-6d78602372b7', 'dg.ANV0/ddd1a258-99f4-4e53-b60c-43b470a49834'), {'subject_id': 0, 'drsID': 1}),
 Row(('175e71c9-62aa-4947-8ff9-bac4f912f323', 'dg.ANV0/06308d6e-da39-4af7-b1bb-3d80920d783d'), {'subject_id': 0, 'drsID': 1}),
 Row(('175e71c9-62aa-4947-8ff9-bac4f912f323', 'dg.ANV0/79e36910-51fc-4a3a-a41b-84ad5006d52d'), {'subject_id': 0, 'drsID': 1})]

In [2]:
drsClient = anvilDRSClient('~/.keys/anvil_credentials.json', access_id='s3')
drsClient.getObject(res[2][1])

{'access_methods': [{'access_id': 'gs',
   'access_url': {'url': 'gs://fc-secure-ff8156a3-ddf3-42e4-9211-0fd89da62108/GTEx_Analysis_2017-06-05_v8_ChIPseq_BAM_files/GTEX-1LGOU-5007-SM-AN49H.H3K27ac.GRCh37.duplicate_marked.bam'},
   'region': '',
   'type': 'gs'},
  {'access_id': 's3',
   'access_url': {'url': 's3://AnVIL/GTEx_Analysis_2017-06-05_v8_ChIPseq_BAM_files/GTEX-1LGOU-5007-SM-AN49H.H3K27ac.GRCh37.duplicate_marked.bam'},
   'region': '',
   'type': 's3'}],
 'aliases': [],
 'checksums': [{'checksum': '34c053550bcd22833a3acc64934a138c',
   'type': 'md5'}],
 'contents': [],
 'created_time': '2020-07-08T17:45:39.829456',
 'description': None,
 'form': 'object',
 'id': 'dg.ANV0/79e36910-51fc-4a3a-a41b-84ad5006d52d',
 'mime_type': 'application/json',
 'name': 'GTEX-1LGOU-5007-SM-AN49H.H3K27ac.GRCh37.duplicate_marked.bam',
 'self_uri': 'drs://gen3.theanvil.io/dg.ANV0/79e36910-51fc-4a3a-a41b-84ad5006d52d',
 'size': 3765705144,
 'updated_time': '2020-07-08T17:45:39.829462',
 'version': '

In [3]:
url = drsClient.getAccessURL(res[2][1])

In [4]:
settings = faspRunner.settings
wesClient = sbcgcWESClient(settings['SevenBridgesProject'])

faspRunner.configure(searchClient, drsClient, wesClient)

faspRunner.runQuery(query, 'Anvil GTEX Test from notebook')

Running query
select su.id subject_id, sq.object_id drsID 
from `isbcgc-216220.gtex8.sequencing` sq 
join `isbcgc-216220.gtex8.sample` sa on sq.parent_id = sa.id 
join `isbcgc-216220.gtex8.subject` su on sa.parent_id = su.id 
where su.body_mass_index between 22.0 and 25.0 
and data_type = 'Aligned Reads' and data_format = 'bam'
limit 3
subject=1653e867-769a-4f4b-a17a-6d78602372b7, drsID=dg.ANV0/ddd1a258-99f4-4e53-b60c-43b470a49834
https://cgc-ga4gh-api.sbgenomics.com/ga4gh/wes/v1
workflow submitted, run:df73a45d-5ec3-4304-b86a-558363f81f3c
____________________________________________________________
subject=175e71c9-62aa-4947-8ff9-bac4f912f323, drsID=dg.ANV0/06308d6e-da39-4af7-b1bb-3d80920d783d
https://cgc-ga4gh-api.sbgenomics.com/ga4gh/wes/v1
workflow submitted, run:e8269863-2d02-43a2-9ef3-7f669ec6286d
____________________________________________________________
subject=175e71c9-62aa-4947-8ff9-bac4f912f323, drsID=dg.ANV0/79e36910-51fc-4a3a-a41b-84ad5006d52d
https://cgc-ga4gh-api.sbgen

[{'subject': '1653e867-769a-4f4b-a17a-6d78602372b7',
  'run_id': 'df73a45d-5ec3-4304-b86a-558363f81f3c'},
 {'subject': '175e71c9-62aa-4947-8ff9-bac4f912f323',
  'run_id': 'e8269863-2d02-43a2-9ef3-7f669ec6286d'},
 {'subject': '175e71c9-62aa-4947-8ff9-bac4f912f323',
  'run_id': '9b38b48b-b19d-4ce4-b161-c664b9ee0617'}]